In [19]:
import pandas as pd
import numpy as np
import re

from torch import nn
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import *
import torchvision.models as models


import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from torch.nn import Transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# import pre-processed data
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap

In [27]:
from text_process import process_text
training_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/train.csv')
val_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/val.csv')
testing_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/test.csv')
training_dataset, val_dataset, testing_dataset = process_text(training_dataset, val_dataset, testing_dataset)

In [ ]:
dist_model = SentenceTransformer('stsb-distilbert-base')
dist_model.max_seq_length = 128

train_titles = training_dataset.proc_title.tolist()
train_embed = dist_model.encode(train_titles, show_progress_bar=True, convert_to_tensor=True)

val_titles = val_dataset.proc_title.tolist()
val_embed = dist_model.encode(val_titles, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 172/172 [00:17<00:00,  9.81it/s]


In [ ]:
IMG_SIZE = 224
size = (IMG_SIZE,IMG_SIZE)
img_model = tf.keras.applications.MobileNet(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet' )
def get_imageEmbeddings(model,imagePath):
    image = tf.keras.preprocessing.image.load_img(imagePath,target_size= size)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    img_embeddings = model(input_arr)
    meanImgEmb1 = np.mean(img_embeddings,axis =0)
    meanImgEmb2 = np.mean(meanImgEmb1,axis=0)
    meanImgEmb = np.mean(meanImgEmb2,axis=0)
    return meanImgEmb

In [ ]:
def get_textEmbeddings(model,text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    return text_embedding

In [ ]:

X_train, X_val, y_train, y_val = train_embed, val_embed, training_dataset.label_group, val_dataset.label_group

In [ ]:
text_embeddings={}
image_embeddings={}
for index,row in  training_dataset.iterrows():
    txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
    imagePath = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'+row[1]
    img_emb = get_imageEmbeddings(img_model,imagePath)
    text_embeddings[row[0]] = txt_emb
    image_embeddings[row[0]] = img_emb

In [ ]:
val_text_embeddings={}
val_image_embeddings={}
for index,row in  val_dataset.iterrows():
    txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
    imagePath = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'+row[1]
    img_emb = get_imageEmbeddings(img_model,imagePath)
    val_text_embeddings[row[0]] = txt_emb
    val_image_embeddings[row[0]] = img_emb


In [ ]:
keyList=[]
cembList=[]
imageList=[]
titleList=[]
for index, row in training_dataset.iterrows():
    txt_emb = text_embeddings[row[0]].cpu().numpy()
    img_emb = image_embeddings[row[0]]
    cmb_emb = np.concatenate((txt_emb,img_emb),axis=0)

    norm = np.linalg.norm(cmb_emb)
    cmb_emb_normal = cmb_emb/norm
    
    keyList.append(row[0])
    cembList.append(cmb_emb_normal)
    imageList.append(row['image'])
    titleList.append(row['proc_title'])

In [ ]:
keyList=[]
val_cembList=[]
imageList=[]
titleList=[]
for index, row in val_dataset.iterrows():
    txt_emb = val_text_embeddings[row[0]].cpu().numpy()
    img_emb = val_image_embeddings[row[0]]
    cmb_emb = np.concatenate((txt_emb,img_emb),axis=0)

    norm = np.linalg.norm(cmb_emb)
    cmb_emb_normal = cmb_emb/norm
    
    keyList.append(row[0])
    val_cembList.append(cmb_emb_normal)
    imageList.append(row['image'])
    titleList.append(row['proc_title'])

In [ ]:
X_train, X_val, y_train, y_val = cembList, val_cembList, training_dataset.label_group, val_dataset.label_group


In [22]:
X_val = pd.DataFrame(X_val)
y_val = pd.Series(y_val)

In [23]:

results = []

for k in [1, 2, 3]:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)
    
    # Find misclassified data points
    misclassified_indices = np.where(y_pred != y_val)[0]
    misclassified_data = X_val.iloc[misclassified_indices]
    misclassified_labels = y_val.iloc[misclassified_indices]
    predicted_labels = y_pred[misclassified_indices]
    
    print(f"For k={k}, {len(misclassified_indices)} out of {len(y_val)} data points are misclassified.")
    
    for i in misclassified_indices:
        result = {'K': k, 'True label': y_val.iloc[i], 'Predicted label': y_pred[i]}
        results.append(result)
    
df_results = pd.DataFrame(results)

For k=1, 2509 out of 5480 data points are misclassified.
For k=2, 3179 out of 5480 data points are misclassified.
For k=3, 3408 out of 5480 data points are misclassified.


In [24]:
df_results.head()

,K,True label,Predicted label
0,1,631365480,3019357193
1,1,3651485151,3441184770
2,1,663012346,3151399095
3,1,1761007187,3319810322
4,1,4069659242,2158225477


In [25]:
path_to_csv='/workspaces/Shopee-Price-Match-Guarantee/30_results/Multimodal_missclassified.csv'

In [26]:
df_results.to_csv(path_to_csv, index=False)